# Assignment Week 3 - Part 1

### Importing the needed libraries and request the data from the url:

In [117]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(url)


    
### Creating a soup object and extracting the table part which is relevant

In [99]:
soup = BeautifulSoup(r.text, "html.parser")

table = soup.find("table", attrs = {"class":"wikitable sortable"})
table_data =table.tbody.find_all("tr")
table_data[0:5]

[<tr>
 <th>Postal Code
 </th>
 <th>Borough
 </th>
 <th>Neighbourhood
 </th></tr>,
 <tr>
 <td>M1A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>,
 <tr>
 <td>M2A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>,
 <tr>
 <td>M3A
 </td>
 <td>North York
 </td>
 <td>Parkwoods
 </td></tr>,
 <tr>
 <td>M4A
 </td>
 <td>North York
 </td>
 <td>Victoria Village
 </td></tr>]

### Getting the headers of table: 

In [190]:
headings = []
for th in table_data[0].find_all('th'):
    headings.append(th.text.strip())

headings

['Postal Code', 'Borough', 'Neighbourhood']

### Getting the data of the table: 


In [247]:
final_table = pd.DataFrame()

for tr in table_data[1:]:
    #print("This is tr: ", i)
    j = 0
    row = [0,0,0]
    for td in tr.find_all("td"):
        #print("this is td: ", td.text)
        row[j] = td.text.strip()
        j = j+1
    row =  pd.Series(row)
    final_table = final_table.append(row, ignore_index = True)

# renaming the headers 
final_table.rename(columns = {0: headings[0], 1:headings[1], 2:headings[2]}, inplace = True)
final_table

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


### Cleaning the data 

In [250]:
#Getting the rows for which the borough is not assigned:
UnassignedBoroughs = final_table[final_table["Borough"]=="Not assigned"].index

#Dropping those rows: 
final_table.drop(UnassignedBoroughs, inplace = True)

#Getting the rows for which the Neighborhood is unassigned and setting those equal to the borough: 
for index, row in final_table.iterrows():
    if (row["Neighbourhood"] == "Not assigned"): 
        row["Neighbourhood"] = row["Borough"]

#Group by the postal code
final_table = final_table.groupby("Postal Code").head()
final_table 

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


### Shape of the table

In [249]:
print("The shape of the table is: ", final_table.shape)

The shape of the table is:  (103, 3)


# Assignment week 3 - Part 2!

In [253]:
import geocoder
import wget

Getting the csv file!

In [262]:
#Downloading the file 
url = "http://cocl.us/Geospatial_data"
wget.download(url, "geospatial_coordinates.csv")

'geospatial_coordinates.csv'

In [264]:
#From csv file to pandas dataframe:
Geospatial_data = pd.read_csv('geospatial_coordinates.csv', header = 0)
Geospatial_data.head()

final_table = final_table.merge(Geospatial_data, how = "left", left_on = "Postal Code", right_on = "Postal Code")
final_table

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
